In [7]:
# Import system modules

import os
import arcpy
from arcpy.sa import *
arcpy.env.overwriteOutput = True

# Check out the ArcGIS Spatial Analyst extension license
arcpy.CheckOutExtension("Spatial")

'CheckedOut'

In [8]:
# create definitions for NDWI band math using raster calculator and reclassify to show only water

def sentinelNDWI(Band3,Band8,outImage):
    output_raster = (Band3-Band8)/(Band3+Band8)
    # output_raster = (Raster(output_raster1),[-1,0,0,1],[0, 1])                                       
    output_raster.save(outImage)

def reclassH2O(h2O_Image):
    out_water = arcpy.sa.Reclassify(in_raster=ndwiFullPath,reclass_field="VALUE", remap="-1 0.1 1;0.1 1 NODATA",missing_values="DATA")
    out_water.save(h2O_Image)

def vesselPolygons(h2O_Image, vessel_polygon):
    arcpy.conversion.RasterToPolygon(
        in_raster=h2O_Image,
        out_polygon_features=vessel_polygon,
        simplify="NO_SIMPLIFY",
        raster_field="Value",
        create_multipart_features="SINGLE_OUTER_PART",
        max_vertices_per_feature=None
    )

SyntaxError: invalid syntax (1926543098.py, line 9)

In [ ]:
with arcpy.EnvManager(extent='-8476661.5386 4460363.1808 -8469554.9864 4483736.0627 PROJCS["WGS_1984_Web_Mercator_Auxiliary_Sphere",GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Mercator_Auxiliary_Sphere"],PARAMETER["False_Easting",0.0],PARAMETER["False_Northing",0.0],PARAMETER["Central_Meridian",0.0],PARAMETER["Standard_Parallel_1",0.0],PARAMETER["Auxiliary_Sphere_Type",0.0],UNIT["Meter",1.0]]'):
    out_raster = arcpy.sa.Reclassify(
        in_raster="s1a-iw-grd-vv_clip_threshold.tif",
        reclass_field="Value",
        remap="0 NODATA;1 1",
        missing_values="DATA"
    )
out_raster.save(r"C:\Users\keit8223\Documents\ArcGIS\Projects\Sentinel Anchorage Areas\s1a-iw-grd-vv_clip_threshold_reclass.tif")

In [6]:
Clip_Area = r"C:\Users\keit8223\Documents\ArcGIS\Projects\Sentinel Anchorage Areas\Ancorage_Area.shp"
desc = arcpy.Describe(Clip_Area)

SR = arcpy.Describe(Clip_Area).spatialReference
print(SR)

<geoprocessing spatial reference object object at 0x000002C7DA7F5F70>


In [4]:
xmin = desc.extent.XMin
xmax = desc.extent.XMax
ymin = desc.extent.YMin
ymax = desc.extent.YMax
xmin

-8476661.5386

In [ ]:
# Loop through the folders and get a list of SAFE directories that contain the Sentinel data
# for each directory find the Band 3 and Band 8 images

directory = r'C:\\Users\\keit8223\\Documents\\ArcGIS\\Projects\\Sentinel Anchorage Areas\Data'
outFolder = r'C:\\temp\\'

# Clip_Area = r"C:\Users\keit8223\Documents\ArcGIS\Projects\Sentinel Anchorage Areas\Ancorage_Area.shp"

for folder in os.listdir(directory):
    f = os.path.join(directory, folder)
    if os.path.isdir(f):
        for root, dirs, files in os.walk(f):
            for file in files:
                # print(file)
                if file.endswith("B03.jp2"):
                    ndwiImage = str(file.split('_')[0]) + '_' + str(file.split('_')[1]) + '_NDWI.tif'
                    ndwiFullPath = outFolder + ndwiImage
                    # print(ndwiImage)
                    # print(ndwiFullPath)
                    waterImage = str(file.split('_')[0]) + '_' + str(file.split('_')[1]) + '_h2O.tif'
                    waterFullPath = outFolder + waterImage
                    vesselPoly = str(file.split('_')[0]) + '_' + str(file.split('_')[1]) + '_vessel_poly.shp'
                    vesselPolyPath = outFolder + vesselPoly
                    vesselPoint = str(file.split('_')[0]) + '_' + str(file.split('_')[1]) + '_vessel_point.shp'
                    vesselPointPath = outFolder + vesselPoint
                    B03 = Raster(os.path.join(root, file))
                if file.endswith("B08.jp2"):
                    B08 = Raster(os.path.join(root, file))
                    sentinelNDWI(B03,B08,ndwiFullPath)
                    reclassH2O(waterFullPath)
                    vesselPolygons(waterFullPath,vesselPolyPath)

                    
                    